In [1]:
print("hello world")

hello world


In [1]:
import torch
import torchaudio

Okay so we have to write a custom dataset class no biggie, but the project description says to focus on silence and unknown but there is no such labels 

I guess for unknown you can make argument that special words like marvin, sheila, happy can serve as unknown but not sure about silence.

In [ ]:
whole_dataset = torch.utils.data.DataLoader(
    torchaudio.datasets.ImageFolder(
        "../../data/train/audio", transform=transforms.ToTensor()
    ),
    batch_size=64,
    shuffle=True,
)